In [ ]:
import time
import matplotlib.pyplot as plt
import numpy as np
import platform
import gc

from kyber_wrapper512 import generate_keypair as generate_keypair512, encapsulate as encapsulate512, decapsulate as decapsulate512
from kyber_wrapper768 import generate_keypair as generate_keypair768, encapsulate as encapsulate768, decapsulate as decapsulate768
from kyber_wrapper1024 import generate_keypair as generate_keypair1024, encapsulate as encapsulate1024, decapsulate as decapsulate1024

def benchmark_variant(generate_keypair, encapsulate, decapsulate, iterations=100):

    key_gen_times = []
    encap_times = []
    decap_times = []

    for _ in range(10):
        public_key, secret_key = generate_keypair()
        ciphertext, shared_secret = encapsulate(public_key)
        decapsulate(ciphertext, secret_key)

    for _ in range(iterations):
        start = time.perf_counter_ns()
        public_key, secret_key = generate_keypair()
        end = time.perf_counter_ns()
        key_gen_times.append((end - start) / 1000)  # µs

    ciphertexts = []
    for _ in range(iterations):
        start = time.perf_counter_ns()
        ciphertext, shared_secret = encapsulate(public_key)
        end = time.perf_counter_ns()
        encap_times.append((end - start) / 1000)
        ciphertexts.append(ciphertext)

    for ct in ciphertexts:
        start = time.perf_counter_ns()
        shared_secret = decapsulate(ct, secret_key)
        end = time.perf_counter_ns()
        decap_times.append((end - start) / 1000)

    return {
        "keygen_avg": np.mean(key_gen_times),
        "encap_avg": np.mean(encap_times),
        "decap_avg": np.mean(decap_times)
    }

def log_system_info():
    print("----- System Information -----")
    print(f"Platform: {platform.platform()}")
    print(f"Processor: {platform.processor()}")
    print(f"Python Version: {platform.python_version()}")
    print("------------------------------\n")

def plot_results(results, iterations):
    operations = ['Key Generation', 'Encapsulation', 'Decapsulation']
    variants = ['Kyber-512', 'Kyber-768', 'Kyber-1024']
    
    means = [
        [results['512']['keygen_avg'], results['512']['encap_avg'], results['512']['decap_avg']],
        [results['768']['keygen_avg'], results['768']['encap_avg'], results['768']['decap_avg']],
        [results['1024']['keygen_avg'], results['1024']['encap_avg'], results['1024']['decap_avg']]
    ]

    x = np.arange(len(operations))
    width = 0.25

    fig, ax = plt.subplots(figsize=(10, 6))
    
    bars1 = ax.bar(x - width, means[0], width, label='Kyber-512')
    bars2 = ax.bar(x,         means[1], width, label='Kyber-768')
    bars3 = ax.bar(x + width, means[2], width, label='Kyber-1024')

    ax.set_ylabel('Time (µs)')
    ax.set_title(f'Kyber Benchmark ({iterations} iterations each)')
    ax.set_xticks(x)
    ax.set_xticklabels(operations)
    ax.legend()

    def autolabel(rects):
        for rect in rects:
            height = rect.get_height()
            ax.annotate(f'{height:.1f}',
                        xy=(rect.get_x() + rect.get_width() / 2, height),
                        xytext=(0, 3),
                        textcoords="offset points",
                        ha='center', va='bottom')
    
    autolabel(bars1)
    autolabel(bars2)
    autolabel(bars3)

    plt.tight_layout()
    plt.savefig('kyber_all_variants_performance.png')
    plt.show()

def main():
    iterations = 10000
    gc.disable()  

    log_system_info()

    print("Benchmarking Kyber-512 ...")
    result512 = benchmark_variant(generate_keypair512, encapsulate512, decapsulate512, iterations)
    
    print("Benchmarking Kyber-768 ...")
    result768 = benchmark_variant(generate_keypair768, encapsulate768, decapsulate768, iterations)
    
    print("Benchmarking Kyber-1024 ...")
    result1024 = benchmark_variant(generate_keypair1024, encapsulate1024, decapsulate1024, iterations)
    
    gc.enable()

    results = {
        '512': result512,
        '768': result768,
        '1024': result1024
    }

    # Display results
    print("\n----- Average Performance Results (microseconds) -----")
    for name, res in results.items():
        print(f"Kyber-{name} - KeyGen: {res['keygen_avg']:.2f}, "
              f"Encap: {res['encap_avg']:.2f}, "
              f"Decap: {res['decap_avg']:.2f}")

    plot_results(results, iterations)

if __name__ == "__main__":
    main()
